how much was blank on [date]?
how much was blank from [date1] to [date2]?
how much was blank last quarter?
how much blank currently?
how much recently?
how much over last two quarters?
how much over prior month? <- could imply 30day time period and not explicitly month
how much last month?
compare this year's quarter x to last year/year's.

vaFuncs

"provide all inventory for October of last year."
            
            |
            |
        blackbox
        
- Assumption(We are Identifying Values that we preceive to be values of the DB)

* Process:
    * Eval for Date Cases
        * No Date Cases, Bye
        * Identify Date Case present
    * Execute func that corresponds to date case
        * Understand schema
            * sweep across each of the tables available in a given schema
            * sweep across each of the columns - entities in each table
        * Break user input to form to schema needs
            * Form sub-query if needed
        * form date query
* other approaches 
    * could use datefinder for cases in which date is explicitly defined

In [35]:
import pandas as pd
import re
import joblib

q = "how much was revenue last october?"

class listFuncs(list):
        
        def walkLeft(self, idx=0, dist=0):
            print("walkLeft called\n")
            space=self[:idx]
            print("this is space: ", space)
            if space != []:
                if dist > len(space):
                    dist=dist-len(space)
                    #print("new dist", dist)
                    
                tokens_along_walk=[]
                if dist == 0:
                    return tokens_along_walk
                else:
                    new_idx=idx
                    for i in range(dist):
                        if new_idx >= 0:
                            new_idx-=1
                            #print("this is the new index", new_idx)
                            #print("so this is the term that should be returned", space[new_idx])
                            tokens_along_walk.append(space[new_idx])
                        else:
                            break
                            
                if len(tokens_along_walk)>1:
                    tokens_along_walk.reverse()
                    phrase = [" ".join(tokens_along_walk)+" "+self[idx]]
                else:
                    phrase = [tokens_along_walk[0]+" "+self[idx]]
                print("this is the phrase from walking left", phrase)
                return phrase
            else:
                print("phrase returned from walking left", phrase)
                return []
                
        def walkRight(self, idx=0, dist=0):
            print("walkRight called\n")
            if idx == 0:
                print("nochanges", space)
                space = self
            else:
                space=self[idx+1:]#bottomisinclusive
                print("changes", space)
                
            if space != []:
                print("contents present")
                #there is space
                horizon=len(space)
                
                # calculate available space to walk to horizon
                if dist>dist-((idx+dist)-horizon):
                    print("redefining distance")
                    dist=dist-((idx+dist)-horizon)
                    #print("this is the new distance to walk", dist)
                    
                tokens_along_walk=[]
                if dist == 0:
                    return tokens_along_walk
                else:
                    for i in range(dist):
                        tokens_along_walk.append(space[i])
                    
                phrase = [self[idx]+" "+" ".join(tokens_along_walk)]
                print("phrase returned from walking right", phrase)
                return phrase
            else:
                
                return []
        def regexMatch(self, rterm):
            pass
            #letshavesomefun
            
            
class DateHandler():
    
    def __init__(self, tokens=None, schema=None):
        
        #Q(): Are we currently able to identify the presence of the date?
            #A(): Year, but it is ambiguous
        self.tokens=tokens
        
        self.schema=schema#TODO(): tie in schema to have sense of how to form sql
        
        self.vocab = {"getDate":["on [date]", "for [date]",# link vocab to common date cases, #ex. of getDate: what is the inventory right now?
                                 "of [date]", "where [date]",
                                 "right [date]", "in [date]"
                                ],
                      
                      "getPeriod":["from [date] to [date]", "between [date] and [date]",#ex. of getPeriod: what is was our inventory from april to june?
                                   "period of [date] to [date]", "in [date] to [date]"
                                  ],
                      
                      "getLastDate":["last [date]", "[date] last year", #ex. of getLastDate: what was revenue last october?
                                     "[date] of last year", "last year's [date]"
                                    ], 
                      
                      "getLastPeriod":["last [date]", "prior [date]", "previous [date]"], #ex. of getLastPeriod: what was the budget last year in between october and december?
                      
                      "triggers":["[jJ]an(uary)?", "[fF]eb(ruary)?", "[mM]ar(ch)?", "[aA]pr(il)?", "[mM]ay", "[jJ]un(e)?", #ex. of getMultiple: can you return the budget for fy quarter 2 and 4 in 2017?
                                  "[jJ]ul(y)?", "[aA]ug(ust)?", "[sS]ep(tember)?", "[oO]ct(ober)?", "[nN]ov(ember)?",
                                  "[dD]ec(ember)?","[qA]uarter", "[pP]eriod",
                                  "[dD]uration", "[yY]ear", "[mM]o(nth)?", "[dD]ay", "[dD]ate", "[fF]iscal"
                                 ],
                      "keywords":{
                          "recent":["to this point"],
                          "prior":[],
                      }
                     }
        self.parser_dict = {
            "month":r"(?:[jJ]an*)|(?:[fF]eb*)|(?:[mM]ar*)|(?:[aA]pr*)|(?:[mM]ay*)|(?:[jJ]un*)|(?:[jJ]ul*)|(?:[aA]ug*)|(?:[sS]ep*)|(?:[oO]ct*)|(?:[nN]ov*)|(?:[dD]ec*)",
            "date":{"mmddyys":r"(((\d{2})|(\d{1}))[/.-](\d{2})[/.-]((\d{4})|(\d{2}))$)|(((\d{2})|(\d{1}))[/.-](\d{2})[/.-]((\d{4})|(\d{2}))$)",
                    "mmyys":None,
            },
            "duration":None,
            "period":None,#TODO(): find the diff ways can be ref'd
            "quarter":None#TODO(): find the diff ways can be ref'd
        }
        
        self.funcs={}
    # is coercible to date? how many x in 2012? what department is employee # 2012 in ?
    def getDateEnts(self):    
        print("getDateEnts called")
        try:
            tokens = self.tokens

            trigids = [re.search("|".join(self.vocab["triggers"]), t) for t in tokens]
            
            if any(trigids):
                
                idxs = pd.Series(trigids).dropna().index.tolist()
                date_ents = dict((k.group(), {}) for k in trigids if k)# key: date_ent -> sub-key: phrase, query --> values
                
                for i, date_ent in enumerate(date_ents):
                    idx=idxs[i]
                    date_ents[date_ent]["phrase"], date_ents[date_ent]["case"] = [], []#initialize lists
                    date_ents[date_ent]["phrase"].extend(listFuncs(tokens).walkLeft(idx=idx, dist=2)+listFuncs(tokens).walkRight(idx=idx, dist=2))# construct possible phrases
                
                print("these are the date entities: ", date_ents)
                self.date_ents = date_ents# add attribute
                return self
            else:
                mxs=[re.search(self.parser_dict["date"]["mmddyys"], t.replace("?","")) for t in  tokens]
                print("parser object", self.parser_dict["date"]["mmddyys"])
                print('these are the tokens being passed in ', tokens)
                if not any(mxs):
                    print("imhere2")
                    return("No Date Present")
                else:
                    print("imhere3")
                    date_ents = dict((k.group(), {}) for k in mxs if k)
                    idxs = pd.Series(mxs).dropna().index.tolist()
                    
                    for i, date_ent in enumerate(date_ents):
                        date_ents[date_ent]["phrase"], date_ents[date_ent]["case"] = [], []#initialize lists
                        idx = idxs[i]
                        date_ents[date_ent]["phrase"].extend(listFuncs(tokens).walkLeft(idx=idx, dist=2)+listFuncs(tokens).walkRight(idx=idx, dist=2))# construct possible phrases
                    print("these are the date entities: ", date_ents)
                    self.date_ents = date_ents# add attribute
                return self
        except Exception as e:
            return e

In [41]:
q1="how much was revenue last october?"
q2="what was the inventory on 10/18/2018?" #nocando
q3="for november, what was our average inventory?"
q4="how many employees attritted in the months of october, november, and december?"#  establish a sense of redundancy
q5="what was revenue last quarter?"
q5="what was revenue for this quarter last year?"# establish a sense of hierarchy
q6="what is this month's inventory?"
q7="what is this year's revenue?"
q8="what is the availability to date?"# may need actual regex parsing rather than strict matching
q9="what was the availability at this point last year?"
q10="what is the revenue over the last 5 years?"

## construct json pairing of domain/value pairings/comparison for less than/greater than and migrate the buildquery

DateHandler(tokens=q5.split()).getDateEnts()

getDateEnts called
walkLeft called

this is space:  ['what', 'was', 'revenue', 'for', 'this']
this is the phrase from walking left ['for this quarter']
walkRight called

changes ['last', 'year?']
contents present
redefining distance
phrase returned from walking right ['quarter ']
walkLeft called

this is space:  ['what', 'was', 'revenue', 'for', 'this', 'quarter', 'last']
this is the phrase from walking left ['quarter last year?']
walkRight called

changes []
these are the date entities:  {'quarter': {'phrase': ['for this quarter', 'quarter '], 'query': [], 'case': []}, 'year': {'phrase': ['quarter last year?'], 'query': [], 'case': []}}


In [40]:
#incorporate tests to include the runtime per query
# potential outlier detecton

def evalDateSchema(self):

    try:
        # peak out how schema is currently stored
        entities, tables = joblib.open("curschema.pkl")
        matches = set([k.lower() for k in entities.keys()]).intersection(self.vocab["triggers"])
        # so now we have identified matches from trigger words to the schema. meaning, each of the 
        #trigger words that align with the schema are columns in the dataset that we will have to 
        # use to query. at this point it makes sense to hava a parser associated with each possible 
        # combination and parse according to this using our dictionary of parsers
        if matches:
            #proceed
            if "date" in matches:
                pass
            elif "year" in matches:
                pass
        else:
            pass
        return scheme
    except Exception as e:
        return e
def identifyDateCase(self, tokens):
    try: 

        self.tokens=tokens
        # eval presence of trigger w/in sentence
        self=self.getPhrases().evalPhrases()
        return self
    except Exception as e:
        return e
def evalPhrases(self):
    try:
        match=None
        # for equivalency(?) between the constructs id'd and the user usage
        # if the user usage is in any of our focus areas then we can proceed to implement the func
        # rel to the focus area
        # if there are multiple occurences of the date entities, this will need to occur
        # @ each enity.
        # end goal: construct a sql query to plug in elsewhere
        for date_ent in self.date_ents:
            phrases = self.date_ents[date_ent]["phrase"]
            for case in self.vocab:
                terms=self.vocab[case]
                matches = crossCheck(terms).check(phrases)
                if matches:
                    self.date_ents[date_ent][phrase] = matches
                    self.date_ents[date_ent]["case"]=case
                    break
        return self
    except Exception as e:
        return e

    def getDate(self, idx):
        try:
            # assumes one case - spc'd by idx
            date_ent = self.date_ents[idx]

            # eval if schema stores date in single date col or multiple cols
            # if month, day, year in schema or date, if quarter or other period stored
            self.schema.evalDateSchema()

            pass
        except Exception as e:
            return 
    def getPeriod(self):
        try:
            date_ents = self.date_ents
            schema = self.schema
            pass
        except Exception as e:
            return e
    def getLastDate(self):
        try:
            date_ents = self.date_ents
            schema = self.schema
            pass
        except Exception as e:
            return e
    def getLastPeriod(self):
        try:
            date_ents = self.date_ents
            schema = self.schema
            pass
        except Exception as e:
            return e
        
    
    class crossCheck(list):
        def check(self, other_list):
            try:
                for st in self:
                    for ot in other_list:
                        matches = list(filter(lambda : v != "", re.findall(st, ot)))
                        return matches
            except Exception as e:
                return e
        def parserCheck(self):
            pass


    # got this from a little duckling: https://github.com/facebook/duckling/tree/master/Duckling
    def MMDDRule(self, date_ent):
        pass
    def MMDDYYYYRule(self, date_ent):
        pass
    def quarterTimeRule(self, date_ent):
        pass
    def halfTimeRule(self, date_ent):
        pass
    def fortnightRule(self, date_ent):
        pass
    # pipeline
    # va funcs
    # schema gets passed to the js file
    # output formatting
    # visualization
    # dropdown

SyntaxError: invalid syntax (<ipython-input-40-00234d0d69cb>, line 4)

In [ ]:
DateHandler(tokens=q.split()).getPhrases()

In [ ]:
re.findall(DateHandler().parser_dict["month"], q)

In [ ]:

triggers = ["january", "february", "march", "april", "may", "june", #ex. of getMultiple: can you return the budget for fy quarter 2 and 4 in 2017?
            "july", "august", "september", "october", "november",
            "december", "jan", "feb", "mar", "apr", "jun", "jul",
            "aug", "sep", "oct", "nov", "dec", "quarter", "period",
            "duration", "year", "month", "mon", "tue", "wed", "thur", 
            "fri"
           ]



In [ ]:
DateHandler().crossCheck(q.split()).check(triggers)

In [ ]:
[re.findall(t, q7)[0] for t in DateHandler().vocab["triggers"] if re.findall(t, q7) != []]

ambigdomainclassifier

In [ ]:
import joblib
def writeAlias(self, msg, alias_path):
    
    alias = joblib.load(aliases_path)
    # expex the prior name intially provided and the correction
    #TODO(): form input msg in non-resty way - need to track messages from instantiation to completion for the ambigcdomain engagement
    alias[msg["corrected"]].append(msg["prior"])
    joblib.dump(alias, aliases_path)
    
    
def retrainNER(creds):
    
    NERTrainer(creds).trainNER()


In [ ]:
from IPython.core.display import display, Javascript, HTML

In [ ]:
%%javascript

require.config({
    paths: {
        d3:"https://cdnjs.cloudflare.com/ajax/libs/d3/4.9.1/d3.min.js"
    }
});

(function(element){
    require(['d3'], function(d3) {
        //inlieuofpreformatcontainer
        var svg = d3.select(element.get(0)).append("svg").attr("id", "chart").attr("width", 800).attr("height", 800);
        function createLinearX(data, width){
            var x = d3.scaleLinear().domain([0, 1.05*d3.max(data, (d)=>(+d.value))]).range([0, width]),
                xAxis = d3.axisBottom(x);
            
            return [x, xAxis]
        }
        //TODO(): add capability for time axis
        function createLinearY(data, height){
            var y = d3.scaleLinear().domain([0, 1.05*d3.max(data, (d)=>(+d.value))]).range([heihgt, 0]),
                yAxis=d3.axisLeft(y);
            return [y, yAxis]
        }
        
        function plotLine(data_in){
            var data = preprocessData(data_in),
                margin={top:20, left: 20, right: 20, bottom: 20},
                width=800-margin.left-margin.right,
                height=800-margin.top-margin.bottom,
                X=createLinearX(data, width),
                Y=createLinearY(data, height),
                x=X[0],
                xAxis=X[1],
                y=Y[0],
                yAxis=Y[1];
            
            
        }
    });
})(element)

#### regex meta character breakdown

. matches any single character except a line break
[] matches any character contained between the square brackets
[^] neglects any character matched within the square brackets
* matches 0 or more repetitions of the preceding symbo
+ matches 1 or more repitions of the preceding symbol
? makes the preceding symbol optional to match
{n,m} matches at least n but not more than n reptitions of the preceding symbol
(x,y,z) matches the characters x,y,z in order
| matches either the characters before or the characters after the symbol
\ escapes the next character
^ matches the begininning of the input
$ matches the end of the input